In [1]:
import pandas as pd
import numpy as np
import os 
import networkx as nx
import seaborn as sns
import matplotlib.pyplot as plt
import sys, os

In [2]:
sys.path.insert(1, os.path.join(sys.path[0], '../../../openAi/TS_analysis_TOPIC9/'))

In [3]:
from pyts_utils import proximity_tools
from pyts_utils import descriptive_tools
from sklearn.decomposition import PCA

In [4]:
af=pd.read_csv('antifragilidades_datasets/af.csv')

In [5]:
af.iloc[:,2:].corr()

,af_in_pr,af_in_vl,af_ex_pr,af_ex_vl,af_in_pr2,af_in_vl2,af_ex_pr2,af_ex_vl2
af_in_pr,1.000000,0.671100,-0.047607,-0.011621,0.957643,0.785956,-0.036705,-0.024999
af_in_vl,0.671100,1.000000,-0.008078,0.026799,0.774187,0.959194,-0.003471,0.016606
af_ex_pr,-0.047607,-0.008078,1.000000,0.692871,-0.038237,-0.015104,0.957657,0.802626
af_ex_vl,-0.011621,0.026799,0.692871,1.000000,-0.002612,0.016074,0.789997,0.957202
af_in_pr2,0.957643,0.774187,-0.038237,-0.002612,1.000000,0.823725,-0.028250,-0.014912
af_in_vl2,0.785956,0.959194,-0.015104,0.016074,0.823725,1.000000,-0.009681,0.004890
af_ex_pr2,-0.036705,-0.003471,0.957657,0.789997,-0.028250,-0.009681,1.000000,0.831397
af_ex_vl2,-0.024999,0.016606,0.802626,0.957202,-0.014912,0.004890,0.831397,1.000000


Desde las correlaciones vemos que ex e in contienen informacion distinta matematicamente, sin embargo dado el algoritmo de calculo de antifragilidad, podemos considerar que poseen la misma informacion sobre la antifragilidad de los paises en el tiempo? Entonces nos quedamos solo con ex por tener mas datos. Para conservar la mayor cantidad de informacion de antifragilidad data por volumen o precio, podemos projectar al primer PCA considerando las dimensiones af_ex_pr, af_ex_vl

In [6]:
af = af.rename(columns={'Unnamed: 0':'country','Unnamed: 1':'year'})

In [7]:
af=af.iloc[:,[0,1,4,5,8,9]]

In [8]:
af.iloc[:,2:].corr()

,af_ex_pr,af_ex_vl,af_ex_pr2,af_ex_vl2
af_ex_pr,1.000000,0.692871,0.957657,0.802626
af_ex_vl,0.692871,1.000000,0.789997,0.957202
af_ex_pr2,0.957657,0.789997,1.000000,0.831397
af_ex_vl2,0.802626,0.957202,0.831397,1.000000


In [9]:
toproject_af=af.dropna(axis=0)

In [11]:
pca=PCA(n_components=1).fit(toproject_af.iloc[:,2:])

In [12]:
clean_afs = pca.fit_transform(toproject_af.iloc[:,2:])

In [13]:
pca.explained_variance_ratio_.cumsum()

array([0.87781833])

In [13]:
clean_afs = pd.DataFrame(data=clean_afs,columns=['af_ex_pca'])

In [14]:
clean_afs['country'] = toproject_af.reset_index()['country']
clean_afs['year'] = toproject_af.reset_index()['year']

In [19]:
# clean_afs.to_csv('af_ex_pca.csv')

Tenemos 184 paises importadores y 244 paises exportadores.

Podria ser el caso que los paises que tenemos mas paises exportadores que importadores debido a que los paises que exportan se puede suponer que lo hacen porque les conviene y les sobra los suficiente para ademas poder exportar. Los importadores podemos pensar que importan porque lo necesitan? Podria significar que los paises que no tienen muchos registros como importadores podrian ser los NO-fragiles debido a que no necesitan importar cosas para estar bien?? Los paises que aparecen como importadores pocas veces, aparecen como exportadores muchas veces? Existe esa simetria?? O son asimetricos y pasa que aunque aparescan pocas veces como importadores, tambien aparecen pocos registros de ellos como exportadores??

In [52]:
ex_pr = af.pivot_table(index='year',columns='country',values='af_ex_pr')

In [83]:
from sklearn.preprocessing import power_transform

In [186]:
yeo_johnson=power_transform(ex_pr.dropna(axis=1).T,standardize=False)

/home/rogelio/anaconda3/envs/geopandas_env/lib/python3.7/site-packages/sklearn/preprocessing/_data.py:2995: RuntimeWarning: divide by zero encountered in log
  loglike = -n_samples / 2 * np.log(x_trans.var())


In [187]:
yeo_johnson = pd.DataFrame(data=yeo_johnson,index=ex_pr.dropna(axis=1).columns.values,columns=ex_pr.dropna(axis=1).index)